In [ ]:
# load scalp example
import trimesh
import numpy as np

# scalp surface is from subject 16, session 2 in
# Telesford, Q.K., Gonzalez-Moreira, E., Xu, T. et al. An open-access dataset of naturalistic viewing using simultaneous EEG-fMRI. Sci Data 10, 554 (2023). https://doi.org/10.1038/s41597-023-02458-8

mesh = trimesh.load('outer_skin.stl')
vertices = np.array(mesh.vertices)
faces = np.array(mesh.faces)

In [ ]:
from electrodes_positions.utils.point_picking import pick_fiducials

# pick fiducials using a GUI
picked_points = pick_fiducials(vertices, faces)

In [ ]:
# project the fiducials on the mesh vertices
(RPA, LPA, NAS, IN), (RPA_idx, LPA_idx, NAS_idx, IN_idx) = project_fid_on_mesh(picked_points, vertices, return_positions = True, return_indices=True)

In [ ]:
# creates a standard montage according to the desired system
# see available montages in electrodes_positions.montages._all_montages
from electrodes_positions.montages import create_standard_montage

all_landmarks = create_standard_montage(vertices, faces, fiducials = (RPA_idx, LPA_idx, NAS_idx, IN_idx), system = '10-10', return_indices = False)

In [ ]:
mesh = pv.PolyData(vertices, faces_to_pyvista(faces))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_point_labels(np.array(list(all_landmarks.values())), list(all_landmarks.keys()), render_points_as_spheres = True, point_size = 10, point_color = 'yellow')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 20, point_color = 'green')
plotter.show()

# this is a demo of the coregistration
the position here are from a standard 10-10 montage on a different space (either another patient or a sphere)

In [10]:
Positions = ['-86.0761 -19.9897 -47.9860','85.7939 -20.0093 -48.0310','0.0083 86.8110 -39.9830','-29.4367 83.9171 -6.9900','0.1123 88.2470 -1.7130','29.8723 84.8959 -7.0800','-48.9708 64.0872 -47.6830','-54.8397 68.5722 -10.5900','-45.4307 72.8622 5.9780','-33.7007 76.8371 21.2270','-18.4717 79.9041 32.7520','0.2313 80.7710 35.4170','19.8203 80.3019 32.7640','35.7123 77.7259 21.9560','46.5843 73.8078 6.0340','55.7433 69.6568 -10.7550','50.4352 63.8698 -48.0050','-70.1019 41.6523 -49.9520','-70.2629 42.4743 -11.4200','-64.4658 48.0353 16.9210','-50.2438 53.1112 42.1920','-27.4958 56.9311 60.3420','0.3122 58.5120 66.4620','29.5142 57.6019 59.5400','51.8362 54.3048 40.8140','67.9142 49.8297 16.3670','73.0431 44.4217 -12.0000','72.1141 42.0667 -50.4520','-84.0759 14.5673 -50.4290','-80.7750 14.1203 -11.1350','-77.2149 18.6433 24.4600','-60.1819 22.7162 55.5440','-34.0619 26.0111 79.9870','0.3761 27.3900 88.6680','34.7841 26.4379 78.8080','62.2931 23.7228 55.6300','79.5341 19.9357 24.4380','81.8151 15.4167 -11.3300','84.1131 14.3647 -50.5380','-85.8941 -15.8287 -48.2830','-84.1611 -16.0187 -9.3460','-80.2801 -13.7597 29.1600','-65.3581 -11.6317 64.3580','-36.1580 -9.9839 89.7520','0.4009 -9.1670 100.2440','37.6720 -9.6241 88.4120','67.1179 -10.9003 63.5800','83.4559 -12.7763 29.2080','85.0799 -15.0203 -9.4900','85.5599 -16.3613 -48.2710','-85.6192 -46.5147 -45.7070','-84.8302 -46.0217 -7.0560','-79.5922 -46.5507 30.9490','-63.5562 -47.0088 65.6240','-35.5131 -47.2919 91.3150','0.3858 -47.3180 99.4320','38.3838 -47.0731 90.6950','66.6118 -46.6372 65.5800','83.3218 -46.1013 31.2060','85.5488 -45.5453 -7.1300','86.1618 -47.0353 -45.8690','-73.0093 -73.7657 -40.9980','-72.4343 -73.4527 -2.4870','-67.2723 -76.2907 28.3820','-53.0073 -78.7878 55.9400','-28.6203 -80.5249 75.4360','0.3247 -81.1150 82.6150','31.9197 -80.4871 76.7160','55.6667 -78.5602 56.5610','67.8877 -75.9043 28.0910','73.0557 -73.0683 -2.5400','73.8947 -74.3903 -41.2200','-54.9104 -98.0448 -35.4650','-54.8404 -97.5279 2.7920','-48.4244 -99.3408 21.5990','-36.5114 -100.8529 37.1670','-18.9724 -101.7680 46.5360','0.2156 -102.1780 50.6080','19.8776 -101.7930 46.3930','36.7816 -100.8491 36.3970','49.8196 -99.4461 21.7270','55.6666 -97.6251 2.7300','54.9876 -98.0911 -35.5410','-29.4134 -112.4490 8.8390','0.1076 -114.8920 14.6570','29.8426 -112.1560 8.8000','-29.8184 -114.5700 -29.2160','0.0045 -118.5650 -23.0780','29.7416 -114.2600 -29.2560','-84.1611 -16.0187 -9.3460','-72.4343 -73.4527 -2.4870','85.0799 -15.0203 -9.4900','73.0557 -73.0683 -2.5400','-86.0761 -44.9897 -67.9860','85.7939 -45.0093 -68.0310','-86.0761 -24.9897 -67.9860','85.7939 -25.0093 -68.0310']
Labels = ['LPA','RPA','Nz','Fp1','Fpz','Fp2','AF9','AF7','AF5','AF3','AF1','AFz','AF2','AF4','AF6','AF8','AF10','F9','F7','F5','F3','F1','Fz','F2','F4','F6','F8','F10','FT9','FT7','FC5','FC3','FC1','FCz','FC2','FC4','FC6','FT8','FT10','T9','T7','C5','C3','C1','Cz','C2','C4','C6','T8','T10','TP9','TP7','CP5','CP3','CP1','CPz','CP2','CP4','CP6','TP8','TP10','P9','P7','P5','P3','P1','Pz','P2','P4','P6','P8','P10','PO9','PO7','PO5','PO3','PO1','POz','PO2','PO4','PO6','PO8','PO10','O1','Oz','O2','O9','Iz','O10','T3','T5','T4','T6','M1','M2','A1','A2']


Positions = list(map(lambda x: x.split(' '), Positions))
Positions = np.array(Positions, dtype = float)

positions_dict = dict(zip(Labels, Positions))
positions_dict['NAS'] = positions_dict['Nz']

In [11]:
# aligns the positions with the head using only the fiducials
positions_dict = transform_fiducials(positions_dict, (RPA, LPA, NAS, IN), scale_y = False, shear_y = True)

In [ ]:
mesh = pv.PolyData(vertices, faces_to_pyvista(faces))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_point_labels(np.array(list(all_landmarks.values())), list(all_landmarks.keys()), render_points_as_spheres = True, point_size = 5, point_color = 'yellow')
plotter.add_point_labels(np.array(list(positions_dict.values())), list(positions_dict.keys()), render_points_as_spheres = True, point_size = 10, point_color = 'blue')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 15, point_color = 'green')
plotter.show()

Widget(value='<iframe src="http://localhost:45111/index.html?ui=P_0x7fc79cb66660_8&reconnect=auto" class="pyvi…

In [ ]:
# projects the positions on the scalp
projected_dict = project_electrodes_on_mesh(positions_dict, vertices, faces)

In [ ]:
mesh = pv.PolyData(vertices, faces_to_pyvista(faces))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_point_labels(np.array(list(all_landmarks.values())), list(all_landmarks.keys()), render_points_as_spheres = True, point_size = 5, point_color = 'yellow')
plotter.add_point_labels(np.array(list(projected_dict.values())), list(projected_dict.keys()), render_points_as_spheres = True, point_size = 10, point_color = 'blue')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 15, point_color = 'green')
plotter.show()

Widget(value='<iframe src="http://localhost:45111/index.html?ui=P_0x7fc79cb66090_9&reconnect=auto" class="pyvi…

In [ ]:
# coregisters the position with the scalp
projected_dict = coregister_to_mesh(vertices, faces, positions_dict, DoF = 7, projection = 'approximate', project_result = True)

In [ ]:
mesh = pv.PolyData(vertices, faces_to_pyvista(faces))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_point_labels(np.array(list(all_landmarks.values())), list(all_landmarks.keys()), render_points_as_spheres = True, point_size = 5, point_color = 'yellow')
plotter.add_point_labels(np.array(list(projected_dict.values())), list(projected_dict.keys()), render_points_as_spheres = True, point_size = 10, point_color = 'blue')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 15, point_color = 'green')
plotter.show()

Widget(value='<iframe src="http://localhost:45111/index.html?ui=P_0x7fc794421e50_11&reconnect=auto" class="pyv…